In [1]:
import pandas as pd

In [2]:
forward_df = pd.read_csv('./data/forward.csv', encoding='utf-8')

In [3]:
forward_df['rating'].describe()

count    920.000000
mean       6.590522
std        0.398388
min        5.820000
25%        6.297500
50%        6.560000
75%        6.840000
max        8.460000
Name: rating, dtype: float64

In [4]:
# 6.7이 반이니깐 train용 데이터는 저 기준으로 자르자

In [5]:
train_f_df = forward_df[forward_df['rating'] >= 6.5]

In [6]:
train_f_df.columns

Index(['Unnamed: 0', 'age', 'asists', 'avgp', 'aw', 'blocks', 'clear',
       'crosses', 'disp', 'drb', 'flag', 'fouled', 'fouls', 'full_time',
       'goals', 'half_time', 'inter', 'keyp', 'league', 'longb', 'mins',
       'motm', 'name', 'off', 'offsides', 'owng', 'player_number', 'position',
       'ps_x', 'ps_y', 'rating', 'red', 'spg', 'tackles', 'tall', 'team_name',
       'thrb', 'unstch', 'weight', 'yel', 'labeled_league'],
      dtype='object')

In [7]:
train_f_df0 = train_f_df.drop(['Unnamed: 0', 'flag', 'full_time', 'half_time', 'mins', 'motm', 'player_number', 'ps_y', 'rating', 'team_name', 'league'], axis=1)
train_f_df0['league'] = train_f_df['league']
train_f_df0['team_name'] = train_f_df['team_name']

In [8]:
train_f_df0 = train_f_df0.fillna(0)

In [9]:
from sklearn.cross_validation import train_test_split

In [10]:
train_f_df0.columns

Index(['age', 'asists', 'avgp', 'aw', 'blocks', 'clear', 'crosses', 'disp',
       'drb', 'fouled', 'fouls', 'goals', 'inter', 'keyp', 'longb', 'name',
       'off', 'offsides', 'owng', 'position', 'ps_x', 'red', 'spg', 'tackles',
       'tall', 'thrb', 'unstch', 'weight', 'yel', 'labeled_league', 'league',
       'team_name'],
      dtype='object')

In [11]:
train_f_df1 = train_f_df0.drop(['league', 'team_name', 'name', 'position'], axis=1)

In [12]:
training_data, testing_data = train_test_split(train_f_df1.get_values(), train_size=0.7)

In [13]:
X_training_data = training_data[:,:-1]
y_training_data = training_data[:, -1]
y_training_data = y_training_data.reshape(361, 1)

In [15]:
import numpy as np

In [16]:
np.shape(y_training_data)

(361, 1)

In [137]:
X_testing_data = testing_data[:,:-1]
y_testing_data = testing_data[:, -1]

In [138]:
X_training_data.shape[1]

27

In [17]:
# modeling with tensorflow

import tensorflow as tf
import numpy as np

In [79]:
ITERATIONS = 2000
LEARNING_RATE = 1e-8

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Train the model
feature_count = X_training_data.shape[1]
x = tf.placeholder('float', shape=[None, feature_count], name='x')
y_ = tf.placeholder('float', shape=[None, 1], name='y_')

print(x.get_shape())

nodes = 20

w1 = weight_variable([feature_count, nodes])
b1 = bias_variable([nodes])
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w2 = weight_variable([nodes, 1])
b2 = bias_variable([1])
y = tf.nn.sigmoid(tf.matmul(l1, w2) + b2)

cross_entropy = tf.reduce_mean(y_*tf.log(tf.maximum(0.00001, y)) + (1.0 - y_)*tf.log(tf.maximum(0.00001, 1.0-y)))
reg = 0.01 * (tf.reduce_mean(tf.square(w1)) + tf.reduce_mean(tf.square(w2)))

predict = (y > 0.5)

correct_prediction = tf.equal(predict, (y_ > 0.5))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))


train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy + reg)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(ITERATIONS):
    feed={x:X_training_data, y_:y_training_data}
    sess.run(train_step, feed_dict=feed)
    if i % 1000 == 0 or i == ITERATIONS -1:
        print('{} {} {:.2f}%'.format(i, sess.run(cross_entropy, feed_dict=feed), sess.run(accuracy, feed_dict=feed)*100))
        

(?, 27)
0 5.306044578552246 85.32%
1000 5.296889305114746 85.32%
1999 5.287741661071777 85.32%


In [78]:
ITERATIONS = 2000
LEARNING_RATE = 1e-8

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Train the model
feature_count = X_training_data.shape[1]
x = tf.placeholder('float', shape=[None, feature_count], name='x')
y_ = tf.placeholder('float', shape=[None, 1], name='y_')

print(x.get_shape())

nodes = 20

w1 = weight_variable([feature_count, nodes])
b1 = bias_variable([nodes])
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w2 = weight_variable([nodes, 1])
b2 = bias_variable([1])
y = tf.nn.sigmoid(tf.matmul(l1, w2) + b2)

cross_entropy = tf.reduce_mean(y_*tf.log(tf.maximum(0.0000001, y)) + (1.0 - y_)*tf.log(tf.maximum(0.0000001, 1.0-y)))
reg = 0.01 * (tf.reduce_mean(tf.square(w1)) + tf.reduce_mean(tf.square(w2)))

predict = (y > 0.5)

correct_prediction = tf.equal(predict, (y_ > 0.5))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))


train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy + reg)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(ITERATIONS):
    feed={x:X_training_data, y_:y_training_data}
    sess.run(train_step, feed_dict=feed)
    if i % 1000 == 0 or i == ITERATIONS -1:
        print('{} {} {:.2f}%'.format(i, sess.run(cross_entropy, feed_dict=feed), sess.run(accuracy, feed_dict=feed)*100))
        

(?, 27)
0 8.310705184936523 85.32%
1000 8.302206993103027 85.32%
1999 8.293720245361328 85.32%


In [53]:
ITERATIONS = 2000
LEARNING_RATE = 1e-8

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Train the model
feature_count = X_training_data.shape[1]
x = tf.placeholder('float', shape=[None, feature_count], name='x')
y_ = tf.placeholder('float', shape=[None, 1], name='y_')

print(x.get_shape())

nodes = 20

w1 = weight_variable([feature_count, nodes])
b1 = bias_variable([nodes])
l1 = tf.nn.relu(tf.matmul(x, w1) + b1)

w2 = weight_variable([nodes, 1])
b2 = bias_variable([1])
y = tf.nn.sigmoid(tf.matmul(l1, w2) + b2)

cross_entropy = tf.reduce_mean(y_*tf.log(tf.maximum(0.00001, y)) + (1.0 - y_)*tf.log(tf.maximum(0.00001, 1.0-y)))
reg = 0.01 * (tf.reduce_mean(tf.square(w1)) + tf.reduce_mean(tf.square(w2)))

predict = (y > 0.5)

correct_prediction = tf.equal(predict, (y_ > 0.5))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))


train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy + reg)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(ITERATIONS):
    feed={x:X_training_data, y_:y_training_data}
    sess.run(train_step, feed_dict=feed)
    if i % 1000 == 0 or i == ITERATIONS -1:
        print('{} {} {:.2f}%'.format(i, sess.run(cross_entropy, feed_dict=feed), sess.run(accuracy, feed_dict=feed)*100))
        

(?, 27)
0 16.18036651611328 85.32%
1000 16.172746658325195 85.32%
1999 16.164825439453125 85.32%


### LCY output

In [82]:
total_player_df = pd.read_csv("./merged_player.csv", encoding='utf-8')

In [85]:
LCY = total_player_df[total_player_df['name'] == "Lee Chung-yong"]

In [89]:
LCY0 = LCY.drop(['Unnamed: 0', 'Unnamed: 0.1', 'flag', 'full_time', 'half_time', 'mins', 'motm', 'player_number', 'ps_y', 'rating', 'team_name', 'league', 'position', 'name'], axis=1)

In [92]:
X_LCY = LCY0.get_values()

In [95]:
X_LCY

array([[  27. ,    0. ,    0.5,    0.4,    0. ,    0.2,   85.9,    1.1,
           0.3,    0.5,    0.3,    1. ,    0.5,    0.5,    0.5,    0. ,
           0. ,    0. ,   85.9,    0. ,    0.4,    1.2,  180. ,    0.7,
           0.5,   75. ,    0. ]])